In [1]:
import pandas as pd
import GladUtil as gd

## 수수료 지급율 변경전후 비교

In [84]:
## glad표준 엑셀 파일
# path ='C:/Users/User/Desktop/myga/'
bojong = '손보'
sheet =  '생명보험(회사)' if bojong == '생보' else '손해보험(회사)'

path = 'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/1.원수사지급율/'
in_file1 = f'수수료(2022.1)선지급형_{bojong}.xlsx'  #당월
in_file2 = f'수수료(2021.11)선지급형_{bojong}.xlsx'  #전월
outfile = f'e:/temp/{bojong}수수료변경_2201.xlsx'
c = '회사 보종 상품명 납기 환산율 환산율2 환산율3 1회 수입P1 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 지급계1y 수입p1y 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회 지급계2y 수입P2y 25회 26회 27회 28회 29회 30회 31회 32회 33회 34회 35회 36회 지급계3y 수입P3y 지급계 수입P 비고'
columns = c.split(' ')

def pre_proc(df):
    df['회사'].fillna(method='ffill',inplace=True)
    df['보종'] = df.보종.replace(0,None)
    df['보종'].fillna(method='ffill',inplace=True)
    df.loc[:,'보종'] = df['보종'].str.replace('\n','').str.replace(' ','')
    df.loc[:,'회사'] = df['회사'].str.replace('\n','').str.replace(' ','')
    df.fillna(0,inplace=True )
    df = df[df['지급계']!=0]
    changed = round(df.iloc[:,4:] * 100,2)   # %를 숫자로
    df.loc[:, df.columns[4:]] = changed
    return df


df = pd.read_excel(path+in_file1,sheet_name=sheet,header=4,names=columns,usecols="a:ba")
df_aft = pre_proc(df)

df = pd.read_excel(path+in_file2,sheet_name=sheet,header=4,names=columns,usecols="a:ba")
df_bef = pre_proc(df)

In [85]:
df_join = df_aft.join(df_bef.set_index(['회사','보종','납기'])[['환산율','환산율2','환산율3','1회','지급계2y','지급계3y','지급계']], 
                      on=(['회사','보종','납기']), rsuffix='_변경전')
# df_bef.set_index(['회사','보종','납기'])
df_join['총지급율변동'] = df_join['지급계'] - df_join['지급계_변경전']
df_join['초회지급율변동'] = df_join['1회'] - df_join['1회_변경전']
df_join['2차년지급율변동'] = df_join['지급계2y'] - df_join['지급계2y_변경전']
df_join['3차년지급율변동'] = df_join['지급계3y'] - df_join['지급계3y_변경전']
df_join.fillna(0,inplace=True)

# df_changed = df_join[ (abs(df_join['총지급율변동']) > 0.05) | (abs(df_join['초회지급율변동']) > 0.05)]
exp = f"abs(초회지급율변동) >= 5 or abs(총지급율변동) >= 5 or 지급계_변경전==0"
df_changed = df_join.query(exp).sort_values(['회사','보종','납기'])

#변경전후
df_ch = df_changed[['회사','보종','납기']]
df_bef.insert(0,"gb","변경전")
df_aft.insert(0,"gb","변경후 ---")
df_l1 = pd.merge(df_bef, df_ch, left_on=['회사','보종','납기'], right_on=['회사','보종','납기'], how='inner')
df_l2 = pd.merge(df_aft, df_ch, left_on=['회사','보종','납기'], right_on=['회사','보종','납기'], how='inner')

df_compare3 = pd.concat([df_l1,df_l2]).sort_values(['회사','보종','납기','gb'])

df_all = pd.concat([df_aft,df_bef])
compare_col = ['회사', '보종','납기','1회','지급계']
df_compare2 = df_all.drop_duplicates(subset=compare_col ,keep=False).sort_values(['회사','보종','납기','gb'])

with pd.ExcelWriter(outfile) as writer:
         df_changed.to_excel(writer, sheet_name='변경보종(5%이상)', index=False)
         df_compare2.to_excel(writer, sheet_name="전후비교(1%이상 변동)", index =False)
         df_compare3.to_excel(writer, sheet_name="전후비교(5%이상 변동만)", index =False)
         
# pd.set_option('display.max_columns', 100)
# df_changed

In [ ]:
# -------------------------------------

In [63]:
# drop_duplicates를 이용한 데이터프레임 비교
df_all = pd.concat([df_aft,df_bef])
# df_all.drop_duplicates(subset=df_all.columns.tolist() ,keep=False)
compare_col = ['회사', '보종','납기','1회','지급계']
df_l3 = df_all.drop_duplicates(subset=compare_col ,keep=False).sort_values(['회사','보종','납기','gb'])
w_mode = 'a' if os.path.isfile(outfile) else 'w'  #파일있으면 append mode
with pd.ExcelWriter(outfile, mode = w_mode, engine="openpyxl") as writer:
     df_l3.to_excel(writer, sheet_name='비교2', index=True)

In [ ]:
df=pd.DataFrame({"a":[10,20,30],"b":[50,130,450],"c":["a","bb","cc"]})
dfa=pd.DataFrame({"a":[10,20,30],"b":[51,131,451],"d":["a","bb","cc"],"c":["ad","bdb","dcc"]}, index=[1,2,3])
# cols=["a","b"]
# df[cols].diff()
pd.concat([df,dfa],join='inner', axis=1)

In [ ]:
from functools import reduce
dfs = [df_bef, df_aft]
cols = [*df_bef.columns]
reduce(lambda left,right: pd.merge(left,right,on=cols), dfs)